In [6]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import time

In [7]:
path = "/Users/niharawarawita/Desktop/MSc Project/Data/EMG_data_collection/combined_stats_nihara.csv"
data = pd.read_csv(path)
data.head()

,participant_id,clothes_id,property_id,property_name,interaction_id,rating,rating_level,rating_level_num,sub_window_num,subwindow_start_time,subwindow_end_time,max_ch1_Hand0,max_ch2_Hand0,max_ch3_Hand0,max_ch4_Hand0,max_ch5_Hand0,max_ch6_Hand0,max_ch7_Hand0,max_ch8_Hand0,mean_ch1_Hand0,mean_ch2_Hand0,mean_ch3_Hand0,mean_ch4_Hand0,mean_ch5_Hand0,mean_ch6_Hand0,mean_ch7_Hand0,mean_ch8_Hand0,std_ch1_Hand0,std_ch2_Hand0,std_ch3_Hand0,std_ch4_Hand0,std_ch5_Hand0,std_ch6_Hand0,std_ch7_Hand0,std_ch8_Hand0,max_ch1_Hand1,max_ch2_Hand1,max_ch3_Hand1,max_ch4_Hand1,max_ch5_Hand1,max_ch6_Hand1,max_ch7_Hand1,max_ch8_Hand1,mean_ch1_Hand1,mean_ch2_Hand1,mean_ch3_Hand1,mean_ch4_Hand1,mean_ch5_Hand1,mean_ch6_Hand1,mean_ch7_Hand1,mean_ch8_Hand1,std_ch1_Hand1,std_ch2_Hand1,std_ch3_Hand1,std_ch4_Hand1,std_ch5_Hand1,std_ch6_Hand1,std_ch7_Hand1,std_ch8_Hand1,max_Ax_Hand0,max_Ay_Hand0,max_Az_Hand0,max_Vx_Hand0,max_Vy_Hand0,max_Vz_Hand0,max_Jx_Hand0,max_Jy_Hand0,max_Jz_Hand0,mean_Ax_Hand0,mean_Ay_Hand0,mean_Az_Hand0,mean_Vx_Hand0,mean_Vy_Hand0,mean_Vz_Hand0,mean_Jx_Hand0,mean_Jy_Hand0,mean_Jz_Hand0,std_Ax_Hand0,std_Ay_Hand0,std_Az_Hand0,std_Vx_Hand0,std_Vy_Hand0,std_Vz_Hand0,std_Jx_Hand0,std_Jy_Hand0,std_Jz_Hand0,max_Ax_Hand1,max_Ay_Hand1,max_Az_Hand1,max_Vx_Hand1,max_Vy_Hand1,max_Vz_Hand1,max_Jx_Hand1,max_Jy_Hand1,max_Jz_Hand1,mean_Ax_Hand1,mean_Ay_Hand1,mean_Az_Hand1,mean_Vx_Hand1,mean_Vy_Hand1,mean_Vz_Hand1,mean_Jx_Hand1,mean_Jy_Hand1,mean_Jz_Hand1,std_Ax_Hand1,std_Ay_Hand1,std_Az_Hand1,std_Vx_Hand1,std_Vy_Hand1,std_Vz_Hand1,std_Jx_Hand1,std_Jy_Hand1,std_Jz_Hand1,max_w_Hand0,max_x_Hand0,max_y_Hand0,max_z_Hand0,max_AVx_Hand0,max_AVy_Hand0,max_AVz_Hand0,max_AAx_Hand0,max_AAy_Hand0,max_AAz_Hand0,max_AJx_Hand0,max_AJy_Hand0,max_AJz_Hand0,mean_w_Hand0,mean_x_Hand0,mean_y_Hand0,mean_z_Hand0,mean_AVx_Hand0,mean_AVy_Hand0,mean_AVz_Hand0,mean_AAx_Hand0,mean_AAy_Hand0,mean_AAz_Hand0,mean_AJx_Hand0,mean_AJy_Hand0,mean_AJz_Hand0,std_w_Hand0,std_x_Hand0,std_y_Hand0,std_z_Hand0,std_AVx_Hand0,std_AVy_Hand0,std_AVz_Hand0,std_AAx_Hand0,std_AAy_Hand0,std_AAz_Hand0,std_AJx_Hand0,std_AJy_Hand0,std_AJz_Hand0,max_w_Hand1,max_x_Hand1,max_y_Hand1,max_z_Hand1,max_AVx_Hand1,max_AVy_Hand1,max_AVz_Hand1,max_AAx_Hand1,max_AAy_Hand1,max_AAz_Hand1,max_AJx_Hand1,max_AJy_Hand1,max_AJz_Hand1,mean_w_Hand1,mean_x_Hand1,mean_y_Hand1,mean_z_Hand1,mean_AVx_Hand1,mean_AVy_Hand1,mean_AVz_Hand1,mean_AAx_Hand1,mean_AAy_Hand1,mean_AAz_Hand1,mean_AJx_Hand1,mean_AJy_Hand1,mean_AJz_Hand1,std_w_Hand1,std_x_Hand1,std_y_Hand1,std_z_Hand1,std_AVx_Hand1,std_AVy_Hand1,std_AVz_Hand1,std_AAx_Hand1,std_AAy_Hand1,std_AAz_Hand1,std_AJx_Hand1,std_AJy_Hand1,std_AJz_Hand1
0,19,3,16,softness,119,6,high,3,1,2022-02-13 11:24:11.302,2022-02-13 11:24:17.236,0.293434,0.158296,0.134725,0.123295,0.260187,0.358265,0.151590,0.251419,0.015854,0.010531,0.008259,0.007029,0.009754,0.021309,0.015523,0.019292,0.023753,0.014289,0.009790,0.008177,0.012954,0.030453,0.016137,0.020183,0.199293,0.101873,0.376079,0.216574,0.202599,0.271375,0.242210,0.188400,0.008796,0.006459,0.013203,0.010320,0.008183,0.028418,0.020070,0.014387,0.012807,0.006526,0.017375,0.015932,0.013619,0.030710,0.023665,0.016529,1.250015,0.131363,0.834671,0.185672,1.053368,1.659817,0.668174,1.758989,6.108710,0.815345,-0.059178,0.575591,-0.001551,-0.197908,-0.148105,-0.000346,-0.014641,0.002513,0.093107,0.038298,0.074497,0.021996,0.465519,0.578251,0.061888,0.207504,0.487276,0.857727,0.186783,0.988373,0.039914,3.557402,0.727362,2.758789,1.902613,0.411489,0.649458,0.041346,0.762079,-0.002331,0.230280,-0.183280,0.009433,0.022472,-0.054542,0.104166,0.060691,0.076344,0.007383,0.541964,0.526181,0.164658,0.241989,0.343387,0.272057,0.550485,-0.057506,0.912503,1.349129,1.656573,1.883185,39.494275,35.226491,125.211266,3638.365630,4263.032202,4319.828070,0.235444,0.432982,-0.155555,0.853674,0.025293,0.018932,-0.020493,0.008707,-0.027117,0.463905,-15.888277,-17.566901,-9.093911,0.049834,0.031046,0.021755,0.015185,0.293781,0.376949,0.502582,8.434800,10.977522,13.138149,703.6

In [37]:
# Initialise the random state
num = random.randint(1, 500)
print(f"The generated random seed is {num}") #347

The generated random seed is 177


In [8]:
# Obtain only the physical properties
physical_data = data[data.property_name != 'enjoyment']
physical_data.head()

,participant_id,clothes_id,property_id,property_name,interaction_id,rating,rating_level,rating_level_num,sub_window_num,subwindow_start_time,subwindow_end_time,max_ch1_Hand0,max_ch2_Hand0,max_ch3_Hand0,max_ch4_Hand0,max_ch5_Hand0,max_ch6_Hand0,max_ch7_Hand0,max_ch8_Hand0,mean_ch1_Hand0,mean_ch2_Hand0,mean_ch3_Hand0,mean_ch4_Hand0,mean_ch5_Hand0,mean_ch6_Hand0,mean_ch7_Hand0,mean_ch8_Hand0,std_ch1_Hand0,std_ch2_Hand0,std_ch3_Hand0,std_ch4_Hand0,std_ch5_Hand0,std_ch6_Hand0,std_ch7_Hand0,std_ch8_Hand0,max_ch1_Hand1,max_ch2_Hand1,max_ch3_Hand1,max_ch4_Hand1,max_ch5_Hand1,max_ch6_Hand1,max_ch7_Hand1,max_ch8_Hand1,mean_ch1_Hand1,mean_ch2_Hand1,mean_ch3_Hand1,mean_ch4_Hand1,mean_ch5_Hand1,mean_ch6_Hand1,mean_ch7_Hand1,mean_ch8_Hand1,std_ch1_Hand1,std_ch2_Hand1,std_ch3_Hand1,std_ch4_Hand1,std_ch5_Hand1,std_ch6_Hand1,std_ch7_Hand1,std_ch8_Hand1,max_Ax_Hand0,max_Ay_Hand0,max_Az_Hand0,max_Vx_Hand0,max_Vy_Hand0,max_Vz_Hand0,max_Jx_Hand0,max_Jy_Hand0,max_Jz_Hand0,mean_Ax_Hand0,mean_Ay_Hand0,mean_Az_Hand0,mean_Vx_Hand0,mean_Vy_Hand0,mean_Vz_Hand0,mean_Jx_Hand0,mean_Jy_Hand0,mean_Jz_Hand0,std_Ax_Hand0,std_Ay_Hand0,std_Az_Hand0,std_Vx_Hand0,std_Vy_Hand0,std_Vz_Hand0,std_Jx_Hand0,std_Jy_Hand0,std_Jz_Hand0,max_Ax_Hand1,max_Ay_Hand1,max_Az_Hand1,max_Vx_Hand1,max_Vy_Hand1,max_Vz_Hand1,max_Jx_Hand1,max_Jy_Hand1,max_Jz_Hand1,mean_Ax_Hand1,mean_Ay_Hand1,mean_Az_Hand1,mean_Vx_Hand1,mean_Vy_Hand1,mean_Vz_Hand1,mean_Jx_Hand1,mean_Jy_Hand1,mean_Jz_Hand1,std_Ax_Hand1,std_Ay_Hand1,std_Az_Hand1,std_Vx_Hand1,std_Vy_Hand1,std_Vz_Hand1,std_Jx_Hand1,std_Jy_Hand1,std_Jz_Hand1,max_w_Hand0,max_x_Hand0,max_y_Hand0,max_z_Hand0,max_AVx_Hand0,max_AVy_Hand0,max_AVz_Hand0,max_AAx_Hand0,max_AAy_Hand0,max_AAz_Hand0,max_AJx_Hand0,max_AJy_Hand0,max_AJz_Hand0,mean_w_Hand0,mean_x_Hand0,mean_y_Hand0,mean_z_Hand0,mean_AVx_Hand0,mean_AVy_Hand0,mean_AVz_Hand0,mean_AAx_Hand0,mean_AAy_Hand0,mean_AAz_Hand0,mean_AJx_Hand0,mean_AJy_Hand0,mean_AJz_Hand0,std_w_Hand0,std_x_Hand0,std_y_Hand0,std_z_Hand0,std_AVx_Hand0,std_AVy_Hand0,std_AVz_Hand0,std_AAx_Hand0,std_AAy_Hand0,std_AAz_Hand0,std_AJx_Hand0,std_AJy_Hand0,std_AJz_Hand0,max_w_Hand1,max_x_Hand1,max_y_Hand1,max_z_Hand1,max_AVx_Hand1,max_AVy_Hand1,max_AVz_Hand1,max_AAx_Hand1,max_AAy_Hand1,max_AAz_Hand1,max_AJx_Hand1,max_AJy_Hand1,max_AJz_Hand1,mean_w_Hand1,mean_x_Hand1,mean_y_Hand1,mean_z_Hand1,mean_AVx_Hand1,mean_AVy_Hand1,mean_AVz_Hand1,mean_AAx_Hand1,mean_AAy_Hand1,mean_AAz_Hand1,mean_AJx_Hand1,mean_AJy_Hand1,mean_AJz_Hand1,std_w_Hand1,std_x_Hand1,std_y_Hand1,std_z_Hand1,std_AVx_Hand1,std_AVy_Hand1,std_AVz_Hand1,std_AAx_Hand1,std_AAy_Hand1,std_AAz_Hand1,std_AJx_Hand1,std_AJy_Hand1,std_AJz_Hand1
0,19,3,16,softness,119,6,high,3,1,2022-02-13 11:24:11.302,2022-02-13 11:24:17.236,0.293434,0.158296,0.134725,0.123295,0.260187,0.358265,0.151590,0.251419,0.015854,0.010531,0.008259,0.007029,0.009754,0.021309,0.015523,0.019292,0.023753,0.014289,0.009790,0.008177,0.012954,0.030453,0.016137,0.020183,0.199293,0.101873,0.376079,0.216574,0.202599,0.271375,0.242210,0.188400,0.008796,0.006459,0.013203,0.010320,0.008183,0.028418,0.020070,0.014387,0.012807,0.006526,0.017375,0.015932,0.013619,0.030710,0.023665,0.016529,1.250015,0.131363,0.834671,0.185672,1.053368,1.659817,0.668174,1.758989,6.108710,0.815345,-0.059178,0.575591,-0.001551,-0.197908,-0.148105,-0.000346,-0.014641,0.002513,0.093107,0.038298,0.074497,0.021996,0.465519,0.578251,0.061888,0.207504,0.487276,0.857727,0.186783,0.988373,0.039914,3.557402,0.727362,2.758789,1.902613,0.411489,0.649458,0.041346,0.762079,-0.002331,0.230280,-0.183280,0.009433,0.022472,-0.054542,0.104166,0.060691,0.076344,0.007383,0.541964,0.526181,0.164658,0.241989,0.343387,0.272057,0.550485,-0.057506,0.912503,1.349129,1.656573,1.883185,39.494275,35.226491,125.211266,3638.365630,4263.032202,4319.828070,0.235444,0.432982,-0.155555,0.853674,0.025293,0.018932,-0.020493,0.008707,-0.027117,0.463905,-15.888277,-17.566901,-9.093911,0.049834,0.031046,0.021755,0.015185,0.293781,0.376949,0.502582,8.434800,10.977522,13.138149,703.6

In [12]:
# Obtain only the EMG data
ending_index = 59
emg_data = physical_data.iloc[:, : ending_index]
emg_data.head()

,participant_id,clothes_id,property_id,property_name,interaction_id,rating,rating_level,rating_level_num,sub_window_num,subwindow_start_time,subwindow_end_time,max_ch1_Hand0,max_ch2_Hand0,max_ch3_Hand0,max_ch4_Hand0,max_ch5_Hand0,max_ch6_Hand0,max_ch7_Hand0,max_ch8_Hand0,mean_ch1_Hand0,mean_ch2_Hand0,mean_ch3_Hand0,mean_ch4_Hand0,mean_ch5_Hand0,mean_ch6_Hand0,mean_ch7_Hand0,mean_ch8_Hand0,std_ch1_Hand0,std_ch2_Hand0,std_ch3_Hand0,std_ch4_Hand0,std_ch5_Hand0,std_ch6_Hand0,std_ch7_Hand0,std_ch8_Hand0,max_ch1_Hand1,max_ch2_Hand1,max_ch3_Hand1,max_ch4_Hand1,max_ch5_Hand1,max_ch6_Hand1,max_ch7_Hand1,max_ch8_Hand1,mean_ch1_Hand1,mean_ch2_Hand1,mean_ch3_Hand1,mean_ch4_Hand1,mean_ch5_Hand1,mean_ch6_Hand1,mean_ch7_Hand1,mean_ch8_Hand1,std_ch1_Hand1,std_ch2_Hand1,std_ch3_Hand1,std_ch4_Hand1,std_ch5_Hand1,std_ch6_Hand1,std_ch7_Hand1,std_ch8_Hand1
0,19,3,16,softness,119,6,high,3,1,2022-02-13 11:24:11.302,2022-02-13 11:24:17.236,0.293434,0.158296,0.134725,0.123295,0.260187,0.358265,0.151590,0.251419,0.015854,0.010531,0.008259,0.007029,0.009754,0.021309,0.015523,0.019292,0.023753,0.014289,0.009790,0.008177,0.012954,0.030453,0.016137,0.020183,0.199293,0.101873,0.376079,0.216574,0.202599,0.271375,0.242210,0.188400,0.008796,0.006459,0.013203,0.010320,0.008183,0.028418,0.020070,0.014387,0.012807,0.006526,0.017375,0.015932,0.013619,0.030710,0.023665,0.016529
1,19,3,16,softness,119,6,high,3,2,2022-02-13 11:24:17.266,2022-02-13 11:24:22.230,0.122884,0.133780,0.124893,0.091408,0.131524,0.235000,0.190598,0.290818,0.011650,0.005750,0.006435,0.005890,0.008041,0.017949,0.014696,0.018217,0.015360,0.007237,0.007143,0.006666,0.010937,0.022777,0.016169,0.018938,0.110325,0.043661,0.107576,0.178948,0.225171,0.226867,0.163749,0.161080,0.006305,0.005218,0.010881,0.008229,0.007772,0.025373,0.015664,0.013348,0.007307,0.004489,0.009017,0.010722,0.013539,0.027277,0.016788,0.013117
2,19,3,16,softness,119,6,high,3,3,2022-02-13 11:24:22.250,2022-02-13 11:24:27.250,0.159430,0.060233,0.071761,0.096457,0.219979,0.217755,0.151590,0.172621,0.008972,0.005293,0.006321,0.006410,0.010819,0.020428,0.015314,0.017475,0.012519,0.004666,0.005230,0.007331,0.017249,0.023520,0.016364,0.017805,0.132567,0.064446,0.144146,0.304368,0.234884,0.357279,0.228678,0.106441,0.006942,0.005352,0.011271,0.008961,0.008506,0.027963,0.017513,0.014099,0.007756,0.004686,0.010284,0.013899,0.017021,0.030857,0.018622,0.013271
3,19,3,15,flexibility,120,5,medium,2,1,2022-02-13 11:27:08.930,2022-02-13 11:27:15.106,0.815143,0.452484,0.301881,0.127192,0.155648,0.422347,0.229605,0.428714,0.038270,0.017163,0.015507,0.012461,0.018050,0.033096,0.024128,0.032406,0.061255,0.025264,0.023970,0.014090,0.019126,0.033572,0.024943,0.032842,0.288260,0.392933,0.325735,0.505040,0.267168,0.536120,0.309869,0.243039,0.018098,0.016356,0.027323,0.017402,0.018562,0.057196,0.040356,0.024961,0.022659,0.023071,0.032812,0.026327,0.021832,0.055826,0.038738,0.025950
4,19,3,15,flexibility,120,5,medium,2,2,2022-02-13 11:27:15.175,2022-02-13 11:27:20.095,0.597988,0.174640,0.219251,0.123295,0.171731,0.437959,0.288116,0.251419,0.026383,0.012442,0.011633,0.009940,0.016288,0.037562,0.025824,0.030991,0.044789,0.015000,0.016053,0.010883,0.020198,0.043357,0.032459,0.033718,0.288260,0.143453,0.177709,0.209854,0.525445,0.305736,0.350465,0.243039,0.021661,0.012210,0.020641,0.015450,0.015092,0.043297,0.036047,0.025502,0.027376,0.012963,0.022271,0.019692,0.028912,0.046049,0.044263,0.026861


In [71]:
starting_index = 11
X = emg_data.iloc[:,starting_index:].values
y = emg_data[['property_id', 'rating_level_num']].values
print(f'X.shape {X.shape}')
print(f'y.shape {y.shape}')
print('--------------------------------')

# Split the data into training and testing sets
X_train_property, X_test_property, y_train, y_test = train_test_split(X, y, test_size = 0.15, shuffle=True, random_state = num) 
print(f'X_train_property.shape {X_train_property.shape}')
print(f'X_test_property.shape {X_test_property.shape}')
print(f'y_train.shape {y_train.shape}')
print(f'y_test.shape {y_test.shape}')
print('--------------------------------')

y_train_property, y_train_ranking = np.hsplit(y_train,2)
y_test_property, y_test_ranking = np.hsplit(y_test,2)
print(f'y_train_property.shape {y_train_property.shape}')
print(f'y_test_property.shape {y_test_property.shape}')
print(f'y_train_ranking.shape {y_train_ranking.shape}')
print(f'y_test_ranking.shape {y_test_ranking.shape}')
print('--------------------------------')

X_train_ranking = np.concatenate((X_train_property, y_train_property), axis=1)
X_test_ranking = np.concatenate((X_test_property, y_test_property), axis=1)
print(f'X_train_ranking.shape {X_train_ranking.shape}')
print(f'X_test_ranking.shape {X_test_ranking.shape}')



X.shape (539, 48)
y.shape (539, 2)
--------------------------------
X_train_property.shape (458, 48)
X_test_property.shape (81, 48)
y_train.shape (458, 2)
y_test.shape (81, 2)
--------------------------------
y_train_property.shape (458, 1)
y_test_property.shape (81, 1)
y_train_ranking.shape (458, 1)
y_test_ranking.shape (81, 1)
--------------------------------
X_train_ranking.shape (458, 49)
X_test_ranking.shape (81, 49)


In [ ]:
# Step 1: Build CNN+LSTM to predict the property

In [68]:
a, b = np.hsplit(y_train,2)
b.shape

(458, 1)

In [49]:
y_property = np.reshape(y_property, (y_property.shape[0], 1))
y_rating = np.reshape(y_rating, (y_rating.shape[0], 1))
y_comb = np.concatenate((y_property, y_rating), axis=1)
X_train_property, X_test_property, y_train_property, y_test_property = train_test_split(X_property, y_comb, test_size = 0.15, shuffle=True, random_state = num) 



In [47]:
print(X_property.shape)
print(y_property.shape)
b = 
print(b.shape)

a.shape

(539, 48)
(539,)
(539, 1)


(539, 49)

In [14]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
#from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot



In [23]:
filepath = "/Users/niharawarawita/Downloads/UCI HAR Dataset/train/X_train.txt"
train_X_df = read_csv(filepath, header=None, delim_whitespace=True)
X_train = train_X_df.values

In [24]:
filepath = "/Users/niharawarawita/Downloads/UCI HAR Dataset/train/y_train.txt"
train_y_df = read_csv(filepath, header=None, delim_whitespace=True)
y_train = train_y_df.values

In [25]:
filepath = "/Users/niharawarawita/Downloads/UCI HAR Dataset/test/X_test.txt"
test_X_df = read_csv(filepath, header=None, delim_whitespace=True)
X_test = test_X_df.values

In [26]:
filepath = "/Users/niharawarawita/Downloads/UCI HAR Dataset/test/y_test.txt"
test_y_df = read_csv(filepath, header=None, delim_whitespace=True)
y_test = test_y_df.values

In [27]:
print(X_train.shape) #(7352, 561)
print(y_train.shape) #(7352, 561)
print(X_test.shape) #(7352, 561)
print(y_test.shape) #(7352, 561)

(7352, 561)
(7352, 1)
(2947, 561)
(2947, 1)


In [33]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values
 
# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    loaded = dstack(loaded)
    return loaded
 
# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y
 
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + '/Users/niharawarawita/Downloads/UCI HAR Dataset/')
    print(trainX.shape, trainy.shape)
    # load all test
    testX, testy = load_dataset_group('test', prefix + '/Users/niharawarawita/Downloads/UCI HAR Dataset/')
    print(testX.shape, testy.shape)
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(f'trainX.shape {trainX.shape}')
    print(f'trainy.shape {trainy.shape}')
    print(f'testX.shape {testX.shape}')
    print(f'testy.shape {testy.shape}')
   # print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

# load data
trainX, trainy, testX, testy = load_dataset()

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
trainX.shape (7352, 128, 9)
trainy.shape (7352, 6)
testX.shape (2947, 128, 9)
testy.shape (2947, 6)


In [36]:
def evaluate_model(trainX, trainy, testX, testy):
    # define model
    verbose, epochs, batch_size = 0, 25, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    print(n_timesteps, n_features, n_outputs)
    # reshape data into time steps of sub-sequences
    n_steps, n_length = 4, 32
    print(f'trainX.shape before {trainX.shape}')
    print(f'testX.shape before {testX.shape}')
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    print(f'trainX.shape after {trainX.shape}')
    print(f'testX.shape after {testX.shape}')
    
    # define model
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

score = evaluate_model(trainX, trainy, testX, testy)
score

128 9 6
trainX.shape before (7352, 128, 9)
testX.shape before (2947, 128, 9)
trainX.shape after (7352, 4, 32, 9)
testX.shape after (2947, 4, 32, 9)


0.9151679873466492

In [ ]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values
 
# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    loaded = dstack(loaded)
    return loaded
 
# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y
 
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'HARDataset/')
    print(trainX.shape, trainy.shape)
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'HARDataset/')
    print(testX.shape, testy.shape)
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy
 
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
    # define model
    verbose, epochs, batch_size = 0, 25, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    # reshape data into time steps of sub-sequences
    n_steps, n_length = 4, 32
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    # define model
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

evaluate_model(X_train, y_train, X_test, y_test)
# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))
 
# run an experiment
def run_experiment(repeats=10):
    # load data
    trainX, trainy, testX, testy = load_dataset()
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)
 
# run the experiment
run_experiment()